## Fresh start

In [1]:
from tqdm import tqdm
import pandas as pd
import os
import json
import numpy as np
import datetime
import pytz
import matplotlib.pyplot as plt
import nltk
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
import unicodedata
import re
nltk.download('stopwords')
nltk.download('punkt')
stopwords = nltk.corpus.stopwords.words('english')
from nltk.stem import LancasterStemmer,PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arpit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arpit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
def readData(news_dir):
    data_text = []
    data_published = []
    date = []
    time = []
    site = []
    for folder in os.scandir(news_dir):
        print(os.scandir(news_dir),type(os.scandir(news_dir)))
        if folder.name!=".DS_Store":
            for entry in os.scandir(news_dir+folder.name):
                data = json.load(open(news_dir+folder.name+"/"+entry.name, encoding='utf-8'))
                if data['published']!="":
                    data_published.append(data['published'])
                    date_time = datetime.datetime.strptime(data['published'], "%Y-%m-%dT%H:%M:%S.%f%z").astimezone(pytz.utc)
                    date.append(str(date_time.date()))
                    time.append(str(date_time.time()))
                else:
                    data_published.append("No Value")
                    date.append("No Value")
                    time.append("No Value")
                if data['text']!="":
                    data_text.append(data['text'].lower())
                else:
                    data_text.append("no value")
                    
                if data['thread']['site_full'] != '':
                    site.append(data['thread']['site_full'])
                else:
                    site.append("unknown")
    news_df=pd.DataFrame({'date_time':data_published, 'text':data_text, 'day':date, 'time':time, 'site':site})
    return news_df

In [51]:
news_directory = "../../Data/News/"

news_df = readData(news_directory)

<nt.ScandirIterator object at 0x000001B2B6E6D560> <class 'nt.ScandirIterator'>
<nt.ScandirIterator object at 0x000001B2B6E71CE0> <class 'nt.ScandirIterator'>
<nt.ScandirIterator object at 0x000001B2B6E73860> <class 'nt.ScandirIterator'>
<nt.ScandirIterator object at 0x000001B2B6E727E0> <class 'nt.ScandirIterator'>
<nt.ScandirIterator object at 0x000001B2B6E727E0> <class 'nt.ScandirIterator'>
<nt.ScandirIterator object at 0x000001B2B6E727E0> <class 'nt.ScandirIterator'>
<nt.ScandirIterator object at 0x000001B2B6E727E0> <class 'nt.ScandirIterator'>
<nt.ScandirIterator object at 0x000001B2B6E727E0> <class 'nt.ScandirIterator'>
<nt.ScandirIterator object at 0x000001B2B6E727E0> <class 'nt.ScandirIterator'>
<nt.ScandirIterator object at 0x000001B2B6E727E0> <class 'nt.ScandirIterator'>
<nt.ScandirIterator object at 0x000001B2B6E727E0> <class 'nt.ScandirIterator'>
<nt.ScandirIterator object at 0x000001B2B6E727E0> <class 'nt.ScandirIterator'>
<nt.ScandirIterator object at 0x000001B2B6E727E0> <c

In [5]:
news_df.sort_values(["day", "time"],axis = 0, ascending=[True, True], inplace=True)

In [7]:
def load_stock_price_dataset(path):
    stock_df=pd.read_csv(path,names=['day','time','x1','price','x2','x3','x4'])[['day','time','price']]
    stock_df.day = stock_df.day.apply(lambda s:s.replace('.','-'))
    return stock_df

In [48]:
amazon_stock_price_60 = load_stock_price_dataset('../../Data/CHARTS/AMAZON60.csv')

In [49]:
apple_stock_price_60 = load_stock_price_dataset('../../Data/CHARTS/APPLE60.csv')

In [52]:
def get_label(stocks_df, day, time):
    next_time_step_indexes = stocks_df[(stocks_df.day == day) & (stocks_df.time > time)].index
    if len(next_time_step_indexes) != 0:
        next_time_step_index = next_time_step_indexes[0]
    #    print("pass")
    else:
        return np.nan
        next_time_step_indexes = stocks_df[(stocks_df.day > day)].index
        if len(next_time_step_indexes) != 0:
            next_time_step_index = next_time_step_indexes[0]
    #       print("pass")
        else:
    #       print("fail")    
            return np.nan
    prev_time_step_index = next_time_step_index - 1 if next_time_step_index>0 else 0
    
    label = 1 if stocks_df.iloc[next_time_step_index].price >= stocks_df.iloc[prev_time_step_index].price else 0
    return label


def assign_labels(news_df, stocks_df_1, stocks_df_2):
    labels_df = pd.DataFrame(columns=["text", "label_1", "label_2", "day", "time", "site"])
    for row_index in tqdm(range(len(news_df))):
        row = news_df.iloc[row_index]
        
        day = row.day
        time = row.time
        text = row.text
        
        label_1 = get_label(stocks_df_1, day, time)
        label_2 = get_label(stocks_df_2, day, time)
        
        labels_df.loc[len(labels_df)] = [text, label_1, label_2, day, time, row.site]
        
        
    return labels_df

In [53]:
labels_df = assign_labels(news_df, amazon_stock_price_60, apple_stock_price_60)

100%|████████████████████████████████████████████████████████████████████████████| 78055/78055 [29:44<00:00, 43.74it/s]


In [54]:
def get_sentences(paragraph):
    sentences = []
    
    first_split = sent_tokenize(paragraph)
    
    for maybe_sentences in first_split:
        our_sentences = maybe_sentences.split("\n")
        sentences.extend(our_sentences)
        
    return sentences

In [55]:
labels_df['sentences'] = labels_df.text.apply(get_sentences)

In [56]:
labels_df['n_sentences'] = labels_df.sentences.apply(lambda s_list: len(s_list))

In [57]:
labels_df.n_sentences.describe()

count    78055.000000
mean        41.844533
std         43.179156
min          0.000000
25%         19.000000
50%         35.000000
75%         54.000000
max       1964.000000
Name: n_sentences, dtype: float64

In [58]:
sentences_df = labels_df.drop(columns=["n_sentences","text"]).explode('sentences').reset_index(drop=True)

In [59]:
sentences_df.shape

(3266207, 6)

In [60]:
sentences_df = sentences_df.dropna(subset=['sentences'])

In [61]:
sentences_df.head()

,label_1,label_2,day,time,site,sentences
0,0,0,2018-01-03,20:33:00,www.nephrologynews.com,annual rpa meeting emphasizes physician leader...
1,0,0,2018-01-03,20:33:00,www.nephrologynews.com,the renal physicians association has long held...
2,0,0,2018-01-03,20:33:00,www.nephrologynews.com,"however, many practitioners do not possess ess..."
3,0,0,2018-01-03,20:33:00,www.nephrologynews.com,"to help remedy this situation, the rpa has pla..."
4,0,0,2018-01-03,20:33:00,www.nephrologynews.com,this workshop will be co-facilitated by rpa pr...


In [62]:
def nonAsciiChar(words):
    words_list=[]
    for w in words:
        w=re.sub('[^a-zA-Z]+','',re.sub(r'[\W\d]','',w.lower()))
        format_words=unicodedata.normalize('NFKD', w).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        words_list.append(format_words)
        
    return words_list
def stemWordsRemoval(words):
    stemmer = PorterStemmer()
    words_list=[]
    for word in words:
        word=stemmer.stem(word)
        if word not in words_list:
            words_list.append(word)
    return words_list

def stopWordsRemoval(words):
    words_list=[]
    for w in words:
        if w not in stopwords:
            words_list.append(w)
    return words_list

def removeLinks(words):
    words_list=[]
    for w in words:
        if not re.match('[www]',w):
            words_list.append(w)
    return words_list

def spaceRemoval(words):
    words_list=[]
    for w in words:
        if w!='':
            words_list.append(w)
    return words_list


def dataExtraction(words):
    words=nonAsciiChar(words)
    words=spaceRemoval(words)
    words=stopWordsRemoval(words)
    words=stemWordsRemoval(words)
    words=removeLinks(words)
    return words

In [63]:
def process_sentences(sentences_df):
    sentences_df=sentences_df[sentences_df['sentences'].str.match('^[A-Z a-z 0-9]+')]
    sentences_df=sentences_df.drop_duplicates(keep=False).reset_index(drop=True)
    sentences_df['words'] = sentences_df.sentences.apply(word_tokenize)
    sentences_df['words'] = sentences_df.words.apply(dataExtraction)
    sentences_df['text'] = sentences_df.words.apply(lambda words: " ".join(words))
    sentences_df = sentences_df.drop(columns=["words","sentences"])
    return sentences_df

In [64]:
amazon_words = ['amazon', 'amzn']
amazon_news_df = sentences_df[sentences_df.sentences.str.contains("|".join(amazon_words))]
amazon_news_df = process_sentences(amazon_news_df)

In [65]:
apple_words = ['apple', 'aapl']
apple_news_df = sentences_df[sentences_df.sentences.str.contains("|".join(apple_words))]
apple_news_df = process_sentences(apple_news_df)

In [66]:
amazon_news_df.head()

,label_1,label_2,day,time,site,text
0,NaN,NaN,2018-01-01,08:47:00,finance.yahoo.com,itun huge hit compani viewer migrat increasing...
1,NaN,NaN,2018-01-01,01:29:00,www.financialnewsusa.com,scorecard facebook fb amazon amzn netflix nflx...
2,NaN,NaN,2018-01-01,01:29:00,www.financialnewsusa.com,amazon
3,NaN,NaN,2018-01-01,01:29:00,www.financialnewsusa.com,amazon share increas stock could soon present ...
4,NaN,NaN,2018-01-01,01:29:00,www.financialnewsusa.com,fang stock news quot facebook amazon netflix g...


In [67]:
apple_news_df.shape

(431887, 6)

In [68]:
amazon_labels = amazon_news_df.drop(columns=['label_2'])
amazon_labels = amazon_labels.rename(columns={"label_1":"label"})
amazon_labels = amazon_labels.dropna(subset=['label'])
amazon_labels.reset_index(drop=True, inplace=True)

In [69]:
apple_labels = apple_news_df.drop(columns=['label_1'])
apple_labels = apple_labels.rename(columns={"label_2":"label"})
apple_labels = apple_labels.dropna(subset=['label'])
apple_labels.reset_index(drop=True, inplace = True)

In [70]:
apple_news_df[apple_news_df.label_2.isna()].day.unique()

array(['2018-01-01', '2018-01-03', '2018-01-06', '2018-01-04',
       '2018-01-07', '2018-01-02', '2017-12-31', '2018-01-05',
       '2018-01-08', '2018-01-15', '2018-01-14', '2018-01-13',
       '2018-01-12', '2017-12-30', '2018-01-09', '2018-01-16',
       '2017-12-22', '2018-01-10', '2018-01-18', '2018-01-17',
       '2017-12-08', '2017-12-12', '2018-01-11', '2018-01-19',
       '2018-01-20', '2018-01-21', '2018-01-22', '2018-01-23',
       '2018-01-24', '2018-01-25', '2018-01-27', '2018-01-26',
       '2018-01-28', '2018-01-29', '2018-01-30', '2018-02-01',
       '2018-02-02', '2018-02-03', '2018-02-04', '2018-01-31',
       '2018-02-06', '2018-02-05', '2018-02-10', '2018-02-08',
       '2018-02-11', '2018-02-07', '2018-02-09', '2018-02-12',
       '2018-02-14', '2018-02-13', '2018-02-15', '2018-02-17',
       '2018-02-16', '2018-02-18', '2018-02-19', '2018-02-21',
       '2018-02-20', '2018-02-22', '2018-02-23', '2018-02-24',
       '2018-02-25', '2018-02-26', '2018-02-27', '2018-

In [71]:
amazon_news_df[amazon_news_df.label_1.isna()].day.unique()

array(['2018-01-01', '2018-01-03', '2018-01-06', '2018-01-04',
       '2018-01-02', '2018-01-05', '2018-01-07', '2017-12-31',
       '2018-01-08', '2018-01-15', '2018-01-14', '2018-01-13',
       '2018-01-12', '2018-01-09', '2018-01-16', '2017-12-22',
       '2017-12-30', '2018-01-10', '2018-01-17', '2018-01-21',
       '2018-01-18', '2018-01-20', '2018-01-22', '2018-01-19',
       '2018-01-23', '2018-01-27', '2018-01-26', '2018-01-25',
       '2018-01-24', '2018-01-28', '2018-01-29', '2018-01-30',
       '2018-02-01', '2018-02-03', '2018-02-02', '2018-02-04',
       '2018-01-31', '2018-02-05', '2018-02-08', '2018-02-10',
       '2018-02-09', '2018-02-11', '2018-02-06', '2018-02-12',
       '2018-02-13', '2018-02-19', '2018-02-14', '2018-02-15',
       '2018-02-18', '2018-02-17', '2018-02-23', '2018-02-16',
       '2018-02-21', '2018-02-22', '2018-02-24', '2018-02-25',
       '2018-02-26', '2018-02-27', '2018-02-20', '2018-02-28',
       '2018-03-03', '2018-03-05', '2018-03-04', '2018-

In [72]:
apple_labels.to_csv("../data/processed/full/apple_labelled_60_special.csv", index = False)

In [73]:
amazon_labels.to_csv("../data/processed/full/amazon_labelled_60_special.csv", index = False)

In [34]:
amazon_stock_price_60.iloc[1].day

'2017-03-13'

In [46]:
news_df.iloc[18]

date_time                        2017-12-21T14:27:00.000+02:00
text         there are still plenty of stocks that fit the ...
day                                                 2017-12-21
time                                                  12:27:00
site                                         www.thestreet.com
Name: 485, dtype: object

In [74]:
apple_labels.shape

(298773, 5)